### Import Packages

In [12]:
import utils 
from SDA_layers import StackedDA 
import os
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn import linear_model
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.linear_model import ElasticNet

### Import Dataset

In [3]:
data = pd.read_csv("final_dataset.csv")
patient = data['PatientID']
y = data['Cancer_Type']
X = data.drop(['Cancer_Type','PatientID'],axis=1)



### Test Train Split

In [4]:
X = data.drop(['Cancer_Type','PatientID'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)
le = preprocessing.LabelEncoder()
le.fit(y_train) 
y_train = pd.DataFrame(le.transform(y_train))
y_val = pd.DataFrame(le.transform(y_val))
y_test = pd.DataFrame(le.transform(y_test))


### Validation to determine alpha

In [13]:
regr = linear_model.ElasticNet(max_iter=10000)
alphas = np.logspace(-4, -1, 10)
alphas

array([ 0.0001    ,  0.00021544,  0.00046416,  0.001     ,  0.00215443,
        0.00464159,  0.01      ,  0.02154435,  0.04641589,  0.1       ])

In [14]:
scores = [regr.set_params(alpha=alpha).fit(X_train, y_train).score(X_val, y_val) for alpha in alphas]
best_alpha = alphas[scores.index(max(scores))]


In [15]:
scores

[0.94144374461500169,
 0.93695728466012385,
 0.92891214921657239,
 0.91698419259197272,
 0.90007050904239971,
 0.8752222198020071,
 0.82963405206232566,
 0.7468915233540141,
 0.60275049819073911,
 0.40017464353087184]

In [1]:
best_alpha

NameError: name 'best_alpha' is not defined

### Testing

In [17]:
regr = linear_model.ElasticNet(alpha=best_alpha, max_iter=10000)
model = regr.fit(X_train,y_train)
prediction = model.predict(X_test)
y = y_test[0].as_matrix()
print(accuracy_score(y,prediction.astype(int)))


0.461126005362


In [18]:
from sklearn.metrics import classification_report
print ("Classification Report \n {}".format(classification_report(y, prediction.astype(int))))

Classification Report 
              precision    recall  f1-score   support

          0       0.45      0.94      0.61        36
          1       0.75      0.41      0.53        80
          2       0.65      0.59      0.62        29
          3       0.05      0.17      0.08         6
          4       0.38      0.53      0.44        32
          5       0.45      0.51      0.48        45
          6       0.45      0.48      0.46        52
          7       0.36      0.16      0.22        32
          8       0.15      0.19      0.17        21
          9       0.93      0.33      0.48        40
         10       0.00      0.00      0.00         0

avg / total       0.54      0.46      0.46       373

